In [ ]:
pip install vectorbt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.2/527.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.41.1
    Uninstalling llvmlite-0.41.1:
      Successfully uninstalled llvmlite-0.41.1
  Attempting uninstall: numba
    Found existing installation: numba 0.58.1
    Uninstalling numba-0.58.1:
      Succe

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import vectorbt as vbt

In [ ]:
# prepare data
start = '2019-01-01 UTC'
end = '2020-01-01 UTC'

btc_price = vbt.YFData.download('BTC-USD', start=start, end=end).get('Close')

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [ ]:
btc_price.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 365 entries, 2019-01-01 00:00:00+00:00 to 2019-12-31 00:00:00+00:00
Freq: D
Series name: Close
Non-Null Count  Dtype  
--------------  -----  
365 non-null    float64
dtypes: float64(1)
memory usage: 5.7 KB


In [ ]:
# using Dual Moving Average Crossover strategy

fast_ma = vbt.MA.run(btc_price, 10, short_name='fast')
slow_ma = vbt.MA.run(btc_price, 20, short_name='slow')

entries = fast_ma.ma_crossed_above(slow_ma)
entries

Date
2019-01-01 00:00:00+00:00    False
2019-01-02 00:00:00+00:00    False
2019-01-03 00:00:00+00:00    False
2019-01-04 00:00:00+00:00    False
2019-01-05 00:00:00+00:00    False
                             ...  
2019-12-27 00:00:00+00:00     True
2019-12-28 00:00:00+00:00    False
2019-12-29 00:00:00+00:00    False
2019-12-30 00:00:00+00:00    False
2019-12-31 00:00:00+00:00    False
Freq: D, Length: 365, dtype: bool

In [ ]:
exits = fast_ma.ma_crossed_below(slow_ma)
exits

Date
2019-01-01 00:00:00+00:00    False
2019-01-02 00:00:00+00:00    False
2019-01-03 00:00:00+00:00    False
2019-01-04 00:00:00+00:00    False
2019-01-05 00:00:00+00:00    False
                             ...  
2019-12-27 00:00:00+00:00    False
2019-12-28 00:00:00+00:00    False
2019-12-29 00:00:00+00:00    False
2019-12-30 00:00:00+00:00    False
2019-12-31 00:00:00+00:00    False
Freq: D, Length: 365, dtype: bool

In [ ]:
# portfolio analysis and return
pf = vbt.Portfolio.from_signals(btc_price, entries, exits)
pf.total_return()

0.6351860771192923

Adding one more strategy instance is as simple as adding one more column. Here we are passing an array of window sizes instead of a single value. For each window size in this array, it computes a moving average over the entire price series and stores it in a distinct column.

In [ ]:
fast_ma = vbt.MA.run(btc_price, [10, 20], short_name='fast')
slow_ma = vbt.MA.run(btc_price, [30, 30], short_name='slow')

In [ ]:
entries = fast_ma.ma_crossed_above(slow_ma)
entries

fast_window,10,20
slow_window,30,30
Date,,
2019-01-01 00:00:00+00:00,False,False
2019-01-02 00:00:00+00:00,False,False
2019-01-03 00:00:00+00:00,False,False
2019-01-04 00:00:00+00:00,False,False
2019-01-05 00:00:00+00:00,False,False
...,...,...
2019-12-27 00:00:00+00:00,False,False
2019-12-28 00:00:00+00:00,False,False


In [ ]:
exits = fast_ma.ma_crossed_below(slow_ma)
exits

fast_window,10,20
slow_window,30,30
Date,,
2019-01-01 00:00:00+00:00,False,False
2019-01-02 00:00:00+00:00,False,False
2019-01-03 00:00:00+00:00,False,False
2019-01-04 00:00:00+00:00,False,False
2019-01-05 00:00:00+00:00,False,False
...,...,...
2019-12-27 00:00:00+00:00,False,False
2019-12-28 00:00:00+00:00,False,False


In [ ]:
pf = vbt.Portfolio.from_signals(btc_price, entries, exits)
pf.total_return()

fast_window  slow_window
10           30             0.847151
20           30             0.543411
Name: total_return, dtype: float64

In [ ]:
# Multiple strategy instances and instruments
eth_price = vbt.YFData.download('ETH-USD', start=start, end=end).get('Close')
comb_price = btc_price.vbt.concat(eth_price,
    keys=pd.Index(['BTC', 'ETH'], name='symbol'))
comb_price.vbt.drop_levels(-1, inplace=True)
comb_price

symbol,BTC,ETH
Date,,
2019-01-01 00:00:00+00:00,3843.520020,140.819412
2019-01-02 00:00:00+00:00,3943.409424,155.047684
2019-01-03 00:00:00+00:00,3836.741211,149.135010
2019-01-04 00:00:00+00:00,3857.717529,154.581940
2019-01-05 00:00:00+00:00,3845.194580,155.638596
...,...,...
2019-12-27 00:00:00+00:00,7290.088379,127.214607
2019-12-28 00:00:00+00:00,7317.990234,128.322708
2019-12-29 00:00:00+00:00,7422.652832,134.757980


In [ ]:
fast_ma = vbt.MA.run(comb_price, [10, 20], short_name='fast')
slow_ma = vbt.MA.run(comb_price, [30, 30], short_name='slow')

In [ ]:
entries = fast_ma.ma_crossed_above(slow_ma)
entries

fast_window                   10            20       
slow_window                   30            30       
symbol                       BTC    ETH    BTC    ETH
Date                                                 
2019-01-01 00:00:00+00:00  False  False  False  False
2019-01-02 00:00:00+00:00  False  False  False  False
2019-01-03 00:00:00+00:00  False  False  False  False
2019-01-04 00:00:00+00:00  False  False  False  False
2019-01-05 00:00:00+00:00  False  False  False  False
...                          ...    ...    ...    ...
2019-12-27 00:00:00+00:00  False  False  False  False
2019-12-28 00:00:00+00:00  False  False  False  False
2019-12-29 00:00:00+00:00   True  False  False  False
2019-12-30 00:00:00+00:00  False  False  False  False
2019-12-31 00:00:00+00:00  False  False  False  False

[365 rows x 4 columns]

In [ ]:
exits = fast_ma.ma_crossed_below(slow_ma)
exits

fast_window                   10            20       
slow_window                   30            30       
symbol                       BTC    ETH    BTC    ETH
Date                                                 
2019-01-01 00:00:00+00:00  False  False  False  False
2019-01-02 00:00:00+00:00  False  False  False  False
2019-01-03 00:00:00+00:00  False  False  False  False
2019-01-04 00:00:00+00:00  False  False  False  False
2019-01-05 00:00:00+00:00  False  False  False  False
...                          ...    ...    ...    ...
2019-12-27 00:00:00+00:00  False  False  False  False
2019-12-28 00:00:00+00:00  False  False  False  False
2019-12-29 00:00:00+00:00  False  False  False  False
2019-12-30 00:00:00+00:00  False  False  False  False
2019-12-31 00:00:00+00:00  False  False  False  False

[365 rows x 4 columns]

In [ ]:
pf = vbt.Portfolio.from_signals(comb_price, entries, exits)
pf.total_return()

fast_window  slow_window  symbol
10           30           BTC       0.847151
                          ETH       0.244204
20           30           BTC       0.543411
                          ETH      -0.319102
Name: total_return, dtype: float64

In [ ]:
# visualizaiton of return

mean_return = pf.total_return().groupby('symbol').mean()
mean_return.vbt.barplot(xaxis_title='Symbol', yaxis_title='Mean total return')

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': 'bar',
              'uid': '7fd290d8-5c7b-463f-8017-4ce1ec05d7f7',
              'x': array(['BTC', 'ETH'], dtype=object),
              'y': array([ 0.69528142, -0.03744928])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700,
               'xaxis': {'title': {'text': 'Symbol'}},
               'yaxis': {'title': {'text': 'Mean total return'}}}
})

In [ ]:
symbols = ["BTC-USD", "ETH-USD"]
price = vbt.YFData.download(symbols, missing_index='drop').get('Close')

n = np.random.randint(10, 101, size=1000).tolist()
pf = vbt.Portfolio.from_random_signals(price, n=n, init_cash=100, seed=42)

mean_expectancy = pf.trades.expectancy().groupby(['randnx_n', 'symbol']).mean()
fig = mean_expectancy.unstack().vbt.scatterplot(xaxis_title='randnx_n', yaxis_title='mean_expectancy')
fig.show()

/usr/local/lib/python3.10/dist-packages/vectorbt/data/base.py:527: UserWarning:

Symbols have mismatching index. Dropping missing data points.

